### This notebook is created by Harshall Lamba. 
#### The github link for this project and blog is https://github.com/hlamba28/Word-Level-Eng-Mar-NMT
#### Please refer the above link for detailed explanation



In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [ ]:
lines = pd.read_table("mar.txt",names = ['mar','random'])

In [ ]:
lines.head()

,mar,random
Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [ ]:
# !pip install tensorflow

In [ ]:
lines.reset_index(inplace = True)
lines=lines.drop(["random"], axis=1)
lines=lines.rename(columns={"index": "eng"})

In [ ]:
type(lines)
lines.head()

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [ ]:
lines[:10]

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!
5,Who?,कोण?
6,Wow!,वाह!
7,Fire!,आग!
8,Fire!,फायर!
9,Help!,वाचवा!


In [ ]:
lines.shape

(42702, 2)

In [ ]:
txt = "Hello Sam"
mytable = txt.maketrans("S","P")
print(txt.translate(mytable))


Hello Pam


In [ ]:
digits

'0123456789'

In [ ]:
# Lowercase all the characters
lines.eng = lines.eng.apply(lambda x: x.lower())
lines.mar = lines.mar.apply(lambda x: x.lower())

# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.sample(10)

,eng,mar
16784,money rules the world,START_ पैसा जगावर राज करतो _END
32451,there was silence for a moment,START_ एक क्षण शांतता होती _END
31230,the dog came running up to me,START_ कुत्रा धावतधावत माझ्याकडे आला _END
17314,tom put his fork down,START_ टॉमने आपला काटा खाली ठेवला _END
41033,how long would it take you to paint my garage,START_ माझं गॅरेज रंगवायला तुला किती वेळ लागेल...
33855,youre afraid of me arent you,START_ तू मला घाबरतेस ना _END
34301,school violence is a big problem,START_ शाळेतील हिंसा एक मोठी समस्या आहे _END
3149,i corrected it,START_ मी दुरुस्त केलं _END
25555,all the villagers know him,START_ सगळेच गावकरी त्याला ओळखतात _END
19497,we know where tom went,START_ टॉम कुठे गेला हे आम्हाला माहीत आहे _END


In [ ]:
lines.eng[0]

'go'

In [ ]:
#Vocabulary of english
all_eng_words = set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

#Vocabulary of marathi
all_mar_words = set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_mar_words:
            all_mar_words.add(word)

In [ ]:
all_eng_words 

{'applies',
 'nintendo',
 'remain',
 'extension',
 'remembered',
 'fifteenth',
 'normans',
 'especially',
 'pushups',
 'canadian',
 'morphine',
 'seeing',
 'interest',
 'where',
 'johnny',
 'onto',
 'held',
 'twentyfive',
 'kabul',
 'itch',
 'bottles',
 'dropped',
 'ball',
 'ocean',
 'america',
 'jokes',
 'robot',
 'crash',
 'italianamericans',
 'oil',
 'speak',
 'download',
 'fares',
 'drove',
 'card',
 'behave',
 'spoiled',
 'kenya',
 'king',
 'blue',
 'bench',
 'ottoman',
 'ruin',
 'lifes',
 'newton',
 'possible',
 'clapping',
 'snake',
 'congratulated',
 'landlord',
 'interested',
 'tomll',
 'relative',
 'scientific',
 'ashburton',
 'device',
 'my',
 'human',
 'luck',
 'fathers',
 'others',
 'touched',
 'keyboard',
 'speaking',
 'windows',
 'metals',
 'dressed',
 'artists',
 'gave',
 'fuels',
 'chatting',
 'boulevard',
 'vegetables',
 'refused',
 'sensei',
 'iran',
 'says',
 'insects',
 'chose',
 'goalkeeper',
 'alfred',
 'gravity',
 'rich',
 'weeks',
 'drugstore',
 'heroes',
 'des

In [ ]:
all_mar_words

{'एअरपोर्टला',
 'गेलो',
 'थांबून',
 'कामाबद्दल',
 'प्रश्नावर',
 'शिकवू',
 'आणीन',
 'मूर्खच',
 'भेटवस्तू',
 'सुपरमार्केट',
 'टॉमचे',
 'फोनवरती',
 'शाळेतच',
 'शकशील',
 'व्यक्तीसाठी',
 'ठामपणे',
 'मूर्खासारखा',
 'ठरवेल',
 'टॉमसारखे',
 'आईपेक्षा',
 'फ्लोरेंस',
 'क्षणासाठी',
 'आर्मस्ट्राँग',
 'ताईशी',
 'रस्सा',
 'डॉक्युमेंट',
 'केल्यानंतर',
 'मुलं',
 'तुझ्याशी',
 'वापरून',
 'ताजी',
 'ओठ',
 'कादंबरी',
 'चुकला',
 'आपलीच',
 'आजचे',
 'पोल्क',
 'फोर्ट',
 'राज्याध्यक्ष',
 'ड्युवी',
 'सव्वातीनला',
 'वाटतं',
 'वर्षात',
 'ग्रीसमधील',
 'हेही',
 'इंग्लंडचं',
 'सापडली',
 'फ्राईड',
 'क्लॉस्ट्रोफोबिक',
 'नाकातून',
 'डिस्ट्रिब्यूशन',
 'शेक्सपीअर',
 'अ',
 'स्थानकासमोर',
 'शरीर',
 'इच्छीतो',
 'म्हणूनच',
 'यादीवरून',
 'सुटलो',
 'आत्मा',
 'अ\u200dॅडऑन',
 'ठेवली',
 'वर्ष',
 'खाण',
 'ओळखलंस',
 'पुत्र',
 'फांद्या',
 'टॉवेलची',
 'सिंहाची',
 'रागवलेलास',
 'करण्यासाठी',
 'आलेलं',
 'पाठवलेला',
 'केलेल्या',
 'मानवतेच्या',
 'मॉरोकोची',
 'मेरीकडून',
 'वडीलांना',
 'नियुक्त',
 'दिसला',
 'पोहण्याने',
 'ओसाकामध्ये',
 'मशीन

In [ ]:
#Max length of source sequence
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(" ")))
max_length_src = np.max(length_list)
max_length_src

34

In [ ]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

37

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_mar_words))
num_encoder_tokens =len(all_eng_words)
num_decoder_tokens =len(all_mar_words)
num_encoder_tokens, num_decoder_tokens

(5779, 13975)

In [ ]:
#input_words

In [ ]:
#target_words

In [ ]:
num_decoder_tokens += 1
num_decoder_tokens 

13976

In [ ]:
num_encoder_tokens += 1
num_encoder_tokens 

5780

In [ ]:
input_token_index = dict([(word, i+1) for i , word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i , word in enumerate(target_words)])

In [ ]:
input_token_index

{'a': 1,
 'abbreviation': 2,
 'ability': 3,
 'able': 4,
 'about': 5,
 'above': 6,
 'abroad': 7,
 'absent': 8,
 'absolute': 9,
 'absolutely': 10,
 'absorbed': 11,
 'accent': 12,
 'accept': 13,
 'acceptable': 14,
 'accepted': 15,
 'access': 16,
 'accessible': 17,
 'accident': 18,
 'accidentally': 19,
 'accomplices': 20,
 'according': 21,
 'accordion': 22,
 'account': 23,
 'accused': 24,
 'accusing': 25,
 'accustomed': 26,
 'ace': 27,
 'aces': 28,
 'acetone': 29,
 'ache': 30,
 'aches': 31,
 'achille': 32,
 'achilles': 33,
 'aching': 34,
 'acid': 35,
 'acknowledged': 36,
 'acquired': 37,
 'acres': 38,
 'across': 39,
 'act': 40,
 'acted': 41,
 'acting': 42,
 'action': 43,
 'activist': 44,
 'actor': 45,
 'actors': 46,
 'actress': 47,
 'acts': 48,
 'actual': 49,
 'actually': 50,
 'add': 51,
 'added': 52,
 'addict': 53,
 'addicted': 54,
 'addition': 55,
 'additional': 56,
 'addon': 57,
 'address': 58,
 'adds': 59,
 'administrative': 60,
 'admire': 61,
 'admit': 62,
 'admitted': 63,
 'adopt': 6

In [ ]:
reverse_input_char_index = dict((i,word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i,word) for word,i in target_token_index.items())

In [ ]:
reverse_input_char_index

{1: 'a',
 2: 'abbreviation',
 3: 'ability',
 4: 'able',
 5: 'about',
 6: 'above',
 7: 'abroad',
 8: 'absent',
 9: 'absolute',
 10: 'absolutely',
 11: 'absorbed',
 12: 'accent',
 13: 'accept',
 14: 'acceptable',
 15: 'accepted',
 16: 'access',
 17: 'accessible',
 18: 'accident',
 19: 'accidentally',
 20: 'accomplices',
 21: 'according',
 22: 'accordion',
 23: 'account',
 24: 'accused',
 25: 'accusing',
 26: 'accustomed',
 27: 'ace',
 28: 'aces',
 29: 'acetone',
 30: 'ache',
 31: 'aches',
 32: 'achille',
 33: 'achilles',
 34: 'aching',
 35: 'acid',
 36: 'acknowledged',
 37: 'acquired',
 38: 'acres',
 39: 'across',
 40: 'act',
 41: 'acted',
 42: 'acting',
 43: 'action',
 44: 'activist',
 45: 'actor',
 46: 'actors',
 47: 'actress',
 48: 'acts',
 49: 'actual',
 50: 'actually',
 51: 'add',
 52: 'added',
 53: 'addict',
 54: 'addicted',
 55: 'addition',
 56: 'additional',
 57: 'addon',
 58: 'address',
 59: 'adds',
 60: 'administrative',
 61: 'admire',
 62: 'admit',
 63: 'admitted',
 64: 'adopt

In [ ]:
#target_token_index

In [ ]:
lines = shuffle(lines)
lines.head()

,eng,mar
5435,wheres the map,START_ नकाशा कुठेय _END
24791,the chicken was delicious,START_ चिकन स्वादिष्ट होतं _END
22161,he speaks four languages,START_ ते चार भाषा बोलतात _END
15542,who invented glasses,START_ चष्म्याचा आविष्कार कोणी केला _END
5349,were neighbors,START_ आम्ही शेजारी आहोत _END


In [ ]:
# Train - Test split
X,y = lines.eng, lines.mar
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)
X_train.shape,X_test.shape

((38431,), (4271,))

In [ ]:
X_train

42303    who do you think was watching tv in this room ...
28407                           tom didnt say a word to us
4123                                       did you say yes
9571                                    he works all night
17585                                what do they do there
                               ...                        
11195                                    you cant stop tom
34993                    i found the lost ball in the park
40447           we only stayed in australia for three days
12309                                  leave a blank space
28043                           its been raining for weeks
Name: eng, Length: 38431, dtype: object

#### Save the train and test dataframes for reproducing results later, as they are shuffled 

In [ ]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder model architecture

In [ ]:
latent_dim = 50

In [ ]:
# Encoder 
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero= True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c= encoder_lstm(enc_emb)
# We discard 'encoder_outputs' and only keep the states
encoder_states= [state_h,state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial states
decoder_inputs = Input(shape = (None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# we set up our decoder to return full output sequences,
# and to return internal states as well, We don't use the
# return states in the training  model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim ,  return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,initial_state= encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn 
#`encoder_input_data`, & `decoder_input_data` into decoder_target_data`
model = Model([encoder_inputs, decoder_inputs],decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 1

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 1

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

300/300 [==============================] - 211s 704ms/step - loss: 0.8489 - acc: 0.2105 - val_loss: 0.8301 - val_acc: 0.2276


In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

### Saving weights

In [ ]:
model.save_weights('nmt_weights.h5')

### Loading the weights

In [ ]:
model.load_weights('nmt_weights.h5')

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        
        # Exit condition: either hit max length or find stop token.
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            stop_condition = True
        
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        
        # Update states
        states_value = [h, c]
    
    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:

k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: tom tried on the wig
Actual Marathi Translation:  टॉमने तो विग घालून बघितला 
Predicted Marathi Translation:  मी मी 
